In [ ]:
!pip install openai
!pip install pydantic
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision transformers pillow requests faiss-gpu loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-1g0lyd7t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-1g0lyd7t
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=ccb0927e83e06e0dee379e49d5cfe9acd5a55b4a8f62a5e5fa02957c47844a19
  Stored in directory: /tmp/pip-ephem-wheel-cache-1rtjpzk0/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import json
from pathlib import Path
import sys
from loguru import logger
from pprint import pprint
import re
import time
import os

import requests
import urllib

from pydantic import BaseModel, ValidationError, validator
from typing import Any, Dict, List, Optional

from google.colab import drive

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm import tqdm

import faiss
from transformers import AlignProcessor, AlignModel

# Step 1: Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train_image_path = '/content/drive/My Drive/Building_images_train/'
train_label_path = '/content/drive/My Drive/Building_images_train/train_labels.json'

model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch1/'

test_image_path = '/content/drive/My Drive/Building_images_test_case/'
test_list_path = ["10130743",
                  "10130755",
                  "10130787",
                  "10130809",
                  "10130812",
                  "10130818",
                  "10130838",
                  "10130844",
                  "10130846",
                  "10130851"
                  ]
# model_dir = '/content/drive/My Drive/Image_text models/Clip_fine-tunned_conservative'
# model_dir = '/content/drive/My Drive/Image_text models/Clip_fine-tunned_conservative'


In [ ]:
# Load the JSON file containing image labels
with open(train_label_path, 'r') as f:
    image_labels = json.load(f)

# Create a mapping from labels to images
label_to_images_train = {}
for image, labels in image_labels.items():
    for label in labels:
        label_to_images_train.setdefault(label, set()).add(image)

In [ ]:
# Load the JSON file containing image labels
def load_test_labels(test_path):
    image_labels = {}
    with open(test_path, 'r') as f:
        image_labels = json.load(f)

    # Create a mapping from labels to images
    label_to_images = {}
    for image, labels in image_labels.items():
        for label in labels:
            label_to_images.setdefault(label, set()).add(image)

    # Get the set of all unique labels
    unique_labels = set(label_to_images.keys())

    return label_to_images, unique_labels

In [ ]:
def search_image_finetuned(model, processor, query, image_dir, threshold=4, fix_threshold=0.23):
    """
    Args:
        query (str): The text query to search for.
        image_dir (str): Directory containing the images.
        threshold (float): The percentage drop threshold between consecutive scores.

    Returns:
        List of tuples (image_name, score) and a list of corresponding images.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Prepare the text input
    text_inputs = processor(text=[query], return_tensors="pt", padding=True)
    text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

    # Get text embedding
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # Normalize

    # Compute similarity scores for all images
    image_scores = []
    images = {}

    for image_name in os.listdir(image_dir):
        if image_name.endswith(".json"):
            continue
        image_path = os.path.join(image_dir, image_name)
        image = Image.open(image_path).convert("RGB")

        # Process the image
        image_inputs = processor(images=image, return_tensors="pt")
        image_inputs = {k: v.to(device) for k, v in image_inputs.items()}

        # Get image embedding
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)  # Normalize

        # Compute cosine similarity
        similarity = torch.matmul(text_features, image_features.T).item()
        image_scores.append((image_name, similarity))
        images[image_name] = image

    # Sort by similarity score in descending order
    image_scores = sorted(image_scores, key=lambda x: x[1], reverse=True)

    # Select images based on score drop
    selected_scores = [image_scores[0]]  # Always include the first image
    for i in range(1, len(image_scores)):
        prev_score = image_scores[i - 1][1]
        curr_score = image_scores[i][1]
        percentage_drop = (prev_score - curr_score) / prev_score
        percentage_drop = percentage_drop * 100

        if percentage_drop > threshold:
            break
        elif curr_score > 0.22:
            selected_scores.append(image_scores[i])


    if image_scores[0][1] < fix_threshold:
        logger.info("All the images are bellow threshold")
        selected_scores = []

    # Get the corresponding images
    selected_images = [image_name for image_name, _ in selected_scores]

    return set(selected_images)

In [ ]:
import pandas as pd

def test_result(unique_labels, label_to_images, model, processor, test_image_path, model_name, test_dataset_name, dynamic_threshold):
    total_tp = 0
    total_tn = 0
    total_images = 0
    label_accuracies = {}  # To store accuracies for each label
    all_test_images = set.union(*label_to_images.values())  # Union of all test images

    for label in unique_labels:
        # Ground truth: images that should be retrieved for this label
        true_images = label_to_images[label]

        train_flag = 1
        try:
            train_images = label_to_images_train[label]
        except:
            train_flag = 0

        # Retrieved images from the model
        retrieved_images = search_image_finetuned(
            model=model,
            processor=processor,
            query=label,
            image_dir=test_image_path,
            threshold=dynamic_threshold,
            fix_threshold=0.23
        )

        retrieved_images = set(retrieved_images)  # Ensure it's a set for set operations

        # True Positives (TP): Correctly retrieved images
        tp = true_images & retrieved_images

        # False Positives (FP): Images incorrectly retrieved
        fp = retrieved_images - true_images

        # False Negatives (FN): Images that should have been retrieved but were not
        fn = true_images - retrieved_images

        # True Negatives (TN): Images that are not in the label and not retrieved
        tn = (all_test_images - true_images) - (retrieved_images - true_images)

        # Accuracy calculation including TN
        label_total_images = len(tp) + len(fp) + len(fn) + len(tn)
        label_accuracy = (len(tp) + len(tn)) / label_total_images if label_total_images > 0 else 0

        # Accumulate for total accuracy calculation
        total_tp += len(tp)
        total_tn += len(tn)
        total_images += label_total_images

        # Store label accuracy
        label_accuracies[label] = label_accuracy

        # Print the results for this label
        print(f"Label: {label}")
        if train_flag:
            print(f"Image Count in Train: {len(train_images)}")
        else:
            print(f"Image Count in Train: 0")
        print(f"Image Count in Test: {len(true_images)}")
        print(f"Retrieved Image Count: {len(retrieved_images)}")
        print(f"Accuracy (with TN): {label_accuracy:.2f}")
        print(f"False Positives (FP): {list(fp)}")
        print(f"False Negatives (FN): {list(fn)}")
        print(f"True Negatives (TN): {len(tn)}")
        print("-" * 30)

    # Calculate total accuracy including TN
    total_accuracy = (total_tp + total_tn) / total_images if total_images > 0 else 0

    # Print total accuracy with color
    print("\033[92m" + f"Total Accuracy (with TN): {total_accuracy:.2f}" + "\033[0m")  # Green text

    # Prepare data for Excel
    row_data = {'Model Name': model_name, 'Test Dataset': test_dataset_name}
    row_data.update(label_accuracies)
    row_data['Total Accuracy'] = total_accuracy

    # Return the row data instead of writing to Excel
    return row_data


# CLIP fine-tuned

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold = 4
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 9
Accuracy (with TN): 0.91
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): ['747072340-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 20
------------------------------
Label: washroom
Image Count in Train: 13
Image Count in Test: 1


2024-12-01 08:43:19.914 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: kid's playground
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073239-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 4
Retrieved Image Count: 4
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 7
------------------------------
Label: bed
Image Count in Train: 24
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: exterior
Image Count in Train: 35
Image Count in Test: 5
Retrieved Image Count: 5
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 6
------------------------------


2024-12-01 08:43:20.993 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: balcony
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073235-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: close gym
Image Count in Train: 8
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------


...

Label: play room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 12
------------------------------


2024-12-01 08:43:56.487 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 1
Accuracy (with TN): 0.69
False Positives (FP): ['747073920-800x600.jpeg']
False Negatives (FN): ['747073917-800x600.jpeg', '747073922-800x600.jpeg', '747073921-800x600.jpeg']
True Negatives (TN): 9
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.90
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 6
Accuracy (with TN): 0.75
False Positives (FP): ['747074041-800x600.jpeg', '747074038-800x6

2024-12-01 08:43:59.306 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: lobby
Image Count in Train: 17
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074040-800x600.jpeg', '747074033-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 7
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 5
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074032-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 3
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): ['747074033-800x600.jpeg', '747074040-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 7
------------------------------
Labe

# Conservative Loss

In [ ]:
def search_image_finetuned(model, processor, query, image_dir, threshold=4, fix_threshold=0.23):
    """
    Args:
        query (str): The text query to search for.
        image_dir (str): Directory containing the images.
        threshold (float): The percentage drop threshold between consecutive scores.

    Returns:
        List of tuples (image_name, score) and a list of corresponding images.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Prepare the text input
    text_inputs = processor(text=[query], return_tensors="pt", padding=True)
    text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

    # Get text embedding
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # Normalize

    # Compute similarity scores for all images
    image_scores = []
    images = {}

    for image_name in os.listdir(image_dir):
        if image_name.endswith(".json"):
            continue
        image_path = os.path.join(image_dir, image_name)
        image = Image.open(image_path).convert("RGB")

        # Process the image
        image_inputs = processor(images=image, return_tensors="pt")
        image_inputs = {k: v.to(device) for k, v in image_inputs.items()}

        # Get image embedding
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)  # Normalize

        # Compute cosine similarity
        similarity = torch.matmul(text_features, image_features.T).item()
        image_scores.append((image_name, similarity))
        images[image_name] = image

    # Sort by similarity score in descending order
    image_scores = sorted(image_scores, key=lambda x: x[1], reverse=True)

    # Select images based on score drop
    selected_scores = [image_scores[0]]  # Always include the first image
    for i in range(1, len(image_scores)):
        prev_score = image_scores[i - 1][1]
        curr_score = image_scores[i][1]
        percentage_drop = (prev_score - curr_score) / prev_score
        percentage_drop = percentage_drop * 100

        if percentage_drop > threshold:
            break
        elif curr_score > 0.50:
            selected_scores.append(image_scores[i])


    if image_scores[0][1] < fix_threshold:
        logger.info("All the images are bellow threshold")
        selected_scores = []

    # Get the corresponding images
    selected_images = [image_name for image_name, _ in selected_scores]

    return set(selected_images)

In [ ]:
import pandas as pd

def test_result(unique_labels, label_to_images, model, processor, test_image_path, model_name, test_dataset_name, dynamic_threshold):
    total_tp = 0
    total_tn = 0
    total_images = 0
    label_accuracies = {}  # To store accuracies for each label
    all_test_images = set.union(*label_to_images.values())  # Union of all test images

    for label in unique_labels:
        # Ground truth: images that should be retrieved for this label
        true_images = label_to_images[label]

        train_flag = 1
        try:
            train_images = label_to_images_train[label]
        except:
            train_flag = 0

        # Retrieved images from the model
        retrieved_images = search_image_finetuned(
            model=model,
            processor=processor,
            query=label,
            image_dir=test_image_path,
            threshold=dynamic_threshold,
            fix_threshold=0.5
        )

        retrieved_images = set(retrieved_images)  # Ensure it's a set for set operations

        # True Positives (TP): Correctly retrieved images
        tp = true_images & retrieved_images

        # False Positives (FP): Images incorrectly retrieved
        fp = retrieved_images - true_images

        # False Negatives (FN): Images that should have been retrieved but were not
        fn = true_images - retrieved_images

        # True Negatives (TN): Images that are not in the label and not retrieved
        tn = (all_test_images - true_images) - (retrieved_images - true_images)

        # Accuracy calculation including TN
        label_total_images = len(tp) + len(fp) + len(fn) + len(tn)
        label_accuracy = (len(tp) + len(tn)) / label_total_images if label_total_images > 0 else 0

        # Accumulate for total accuracy calculation
        total_tp += len(tp)
        total_tn += len(tn)
        total_images += label_total_images

        # Store label accuracy
        label_accuracies[label] = label_accuracy

        # Print the results for this label
        print(f"Label: {label}")
        if train_flag:
            print(f"Image Count in Train: {len(train_images)}")
        else:
            print(f"Image Count in Train: 0")
        print(f"Image Count in Test: {len(true_images)}")
        print(f"Retrieved Image Count: {len(retrieved_images)}")
        print(f"Accuracy (with TN): {label_accuracy:.2f}")
        print(f"False Positives (FP): {list(fp)}")
        print(f"False Negatives (FN): {list(fn)}")
        print(f"True Negatives (TN): {len(tn)}")
        print("-" * 30)

    # Calculate total accuracy including TN
    total_accuracy = (total_tp + total_tn) / total_images if total_images > 0 else 0

    # Print total accuracy with color
    print("\033[92m" + f"Total Accuracy (with TN): {total_accuracy:.2f}" + "\033[0m")  # Green text

    # Prepare data for Excel
    row_data = {'Model Name': model_name, 'Test Dataset': test_dataset_name}
    row_data.update(label_accuracies)
    row_data['Total Accuracy'] = total_accuracy

    # Return the row data instead of writing to Excel
    return row_data


In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_conservative/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=10
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 10
Accuracy (with TN): 0.95
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 3
Accuracy (with TN): 0.95
False Positives (FP): ['747072329-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 19
------------------------------
Label: washroom
Image Count in Train: 13
Image Count in Test: 1

2024-12-01 08:45:40.104 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: dining
Image Count in Train: 20
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072578-800x600.jpeg']
True Negatives (TN): 11
------------------------------


2024-12-01 08:45:40.420 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: interior
Image Count in Train: 96
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747072580-800x600.jpeg', '747072578-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: sofa
Image Count in Train: 52
Image Count in Test: 3
Retrieved Image Count: 2
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072580-800x600.jpeg']
True Negatives (TN): 9
------------------------------


...

Label: lobby
Image Count in Train: 17
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074040-800x600.jpeg', '747074033-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 6
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074038-800x600.jpeg']
True Negatives (TN): 5
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 0.83
False Positives (FP): ['747074040-800x600.jpeg']
False Negatives (FN): ['747074032-800x600.jpeg']
True Negatives (TN): 9
------------------------------


2024-12-01 08:46:29.413 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: interior
Image Count in Train: 96
Image Count in Test: 3
Retrieved Image Count: 0
Accuracy (with TN): 0.75
False Positives (FP): []
False Negatives (FN): ['747074032-800x600.jpeg', '747074039-800x600.jpeg', '747074027-800x600.jpeg']
True Negatives (TN): 9
------------------------------
Label: sofa
Image Count in Train: 52
Image Count in Test: 5
Retrieved Image Count: 4
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074027-800x600.jpeg']
True Negatives (TN): 7
------------------------------
Label: TV
Image Count in Train: 25
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 0.83
False Positives (FP): ['747074032-800x600.jpeg']
False Negatives (FN): ['747074039-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 6
Accuracy (with TN): 0.42
False Positives (FP): ['747074041-800x600.jpeg', '747074030-800x600.jpeg', '747074042-800x600.jpeg

2024-12-01 08:46:35.350 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold


Label: kitchen
Image Count in Train: 28
Image Count in Test: 1
Retrieved Image Count: 2
Accuracy (with TN): 0.89
False Positives (FP): ['747074071-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 7
------------------------------
Label: window
Image Count in Train: 41
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074068-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: top view
Image Count in Train: 2
Image Count in Test: 1
Retrieved Image Count: 3
Accuracy (with TN): 0.78
False Positives (FP): ['747074065-800x600.jpeg', '747074063-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 6
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Total Accuracy (with TN): 0.90
All 

# Normal with batch 8

In [ ]:
def search_image_finetuned(model, processor, query, image_dir, threshold=4, fix_threshold=0.23):
    """
    Args:
        query (str): The text query to search for.
        image_dir (str): Directory containing the images.
        threshold (float): The percentage drop threshold between consecutive scores.

    Returns:
        List of tuples (image_name, score) and a list of corresponding images.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Prepare the text input
    text_inputs = processor(text=[query], return_tensors="pt", padding=True)
    text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

    # Get text embedding
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # Normalize

    # Compute similarity scores for all images
    image_scores = []
    images = {}

    for image_name in os.listdir(image_dir):
        if image_name.endswith(".json"):
            continue
        image_path = os.path.join(image_dir, image_name)
        image = Image.open(image_path).convert("RGB")

        # Process the image
        image_inputs = processor(images=image, return_tensors="pt")
        image_inputs = {k: v.to(device) for k, v in image_inputs.items()}

        # Get image embedding
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)  # Normalize

        # Compute cosine similarity
        similarity = torch.matmul(text_features, image_features.T).item()
        image_scores.append((image_name, similarity))
        images[image_name] = image

    # Sort by similarity score in descending order
    image_scores = sorted(image_scores, key=lambda x: x[1], reverse=True)

    # Select images based on score drop
    selected_scores = [image_scores[0]]  # Always include the first image
    for i in range(1, len(image_scores)):
        prev_score = image_scores[i - 1][1]
        curr_score = image_scores[i][1]
        percentage_drop = (prev_score - curr_score) / prev_score
        percentage_drop = percentage_drop * 100

        if percentage_drop > threshold:
            break
        elif curr_score > 0.22:
            selected_scores.append(image_scores[i])


    if image_scores[0][1] < fix_threshold:
        logger.info(f"All the images are bellow threshold, fix_threshold: {fix_threshold}, dynamic threshold: {threshold}")
        selected_scores = []

    # Get the corresponding images
    selected_images = [image_name for image_name, _ in selected_scores]

    return set(selected_images)

In [ ]:
import pandas as pd

def test_result(unique_labels, label_to_images, model, processor, test_image_path, model_name, test_dataset_name, dynamic_threshold):
    total_tp = 0
    total_tn = 0
    total_images = 0
    label_accuracies = {}  # To store accuracies for each label
    all_test_images = set.union(*label_to_images.values())  # Union of all test images

    for label in unique_labels:
        # Ground truth: images that should be retrieved for this label
        true_images = label_to_images[label]

        train_flag = 1
        try:
            train_images = label_to_images_train[label]
        except:
            train_flag = 0

        # Retrieved images from the model
        retrieved_images = search_image_finetuned(
            model=model,
            processor=processor,
            query=label,
            image_dir=test_image_path,
            threshold=dynamic_threshold,
            fix_threshold=0.23
        )

        retrieved_images = set(retrieved_images)  # Ensure it's a set for set operations

        # True Positives (TP): Correctly retrieved images
        tp = true_images & retrieved_images

        # False Positives (FP): Images incorrectly retrieved
        fp = retrieved_images - true_images

        # False Negatives (FN): Images that should have been retrieved but were not
        fn = true_images - retrieved_images

        # True Negatives (TN): Images that are not in the label and not retrieved
        tn = (all_test_images - true_images) - (retrieved_images - true_images)

        # Accuracy calculation including TN
        label_total_images = len(tp) + len(fp) + len(fn) + len(tn)
        label_accuracy = (len(tp) + len(tn)) / label_total_images if label_total_images > 0 else 0

        # Accumulate for total accuracy calculation
        total_tp += len(tp)
        total_tn += len(tn)
        total_images += label_total_images

        # Store label accuracy
        label_accuracies[label] = label_accuracy

        # Print the results for this label
        print(f"Label: {label}")
        if train_flag:
            print(f"Image Count in Train: {len(train_images)}")
        else:
            print(f"Image Count in Train: 0")
        print(f"Image Count in Test: {len(true_images)}")
        print(f"Retrieved Image Count: {len(retrieved_images)}")
        print(f"Accuracy (with TN): {label_accuracy:.2f}")
        print(f"False Positives (FP): {list(fp)}")
        print(f"False Negatives (FN): {list(fn)}")
        print(f"True Negatives (TN): {len(tn)}")
        print("-" * 30)

    # Calculate total accuracy including TN
    total_accuracy = (total_tp + total_tn) / total_images if total_images > 0 else 0

    if

    # Print total accuracy with color
    print("\033[92m" + f"Total Accuracy (with TN): {total_accuracy:.2f}" + "\033[0m")  # Green text

    # Prepare data for Excel
    row_data = {'Model Name': model_name, 'Test Dataset': test_dataset_name}
    row_data.update(label_accuracies)
    row_data['Total Accuracy'] = total_accuracy

    # Return the row data instead of writing to Excel
    return row_data


In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch8/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold = 2.5
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 3
Accuracy (with TN): 0.82
False Positives (FP): ['747072344-800x600.jpeg', '747072339-800x600.jpeg', '747072334-800x600.jpeg']
False Negatives (FN): ['747072336-800x600.jpeg']
True Negatives (TN): 18
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 3
Accuracy (with TN): 0.73
False Positives (FP): []
False Negatives (FN): ['747072331-800x600.jpeg', '747072334-800x600.jpeg', '747072339-800x600.jpeg', '747072333-800x600.jpeg', '747072337-800x600.jpeg', '747072340-800x600.jpeg']
True Negatives (TN): 13
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Ima

2024-12-01 09:10:53.842 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 2


Label: entrance
Image Count in Train: 0
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072339-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 2
Accuracy (with TN): 0.95
False Positives (FP): ['747072325-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 20
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: close gym
Image Count in Train: 8
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: TV
Image Count in Train: 25
Image Count in Test: 1
Retrieved Image Coun

2024-12-01 09:11:08.993 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 2


Label: kid's playground
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073239-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 4
Retrieved Image Count: 1
Accuracy (with TN): 0.73
False Positives (FP): []
False Negatives (FN): ['747073160-800x600.jpeg', '747073235-800x600.jpeg', '747073161-800x600.jpeg']
True Negatives (TN): 7
------------------------------
Label: bed
Image Count in Train: 24
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: exterior
Image Count in Train: 35
Image Count in Test: 5
Retrieved Image Count: 2
Accuracy (with TN): 0.73
False Positives (FP): []
False Negatives (FN): ['747073240-800x600.jpeg', '747073241-800x600.jpeg', '747073239-800x600.jpeg']
Tr

...

Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 3
Accuracy (with TN): 0.69
False Positives (FP): ['747073913-800x600.jpeg', '747073920-800x600.jpeg']
False Negatives (FN): ['747073922-800x600.jpeg', '747073921-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.87
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 7
Accuracy (with TN): 0.67
False Positives (FP): ['747074041-800x600.jpeg', '747074038-800x6

2024-12-01 09:11:48.406 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 2


Label: lobby
Image Count in Train: 17
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074040-800x600.jpeg', '747074033-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074041-800x600.jpeg', '747074038-800x600.jpeg']
True Negatives (TN): 5
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074032-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 3
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): ['747074033-800x600.jpeg', '747074040-800x600.jpeg']
False Negatives (FN): []
True Neg

2024-12-01 09:11:55.435 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 2


Label: top view
Image Count in Train: 2
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074064-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Total Accuracy (with TN): 0.96
All results written to CLIP_fine-tunned_batch8_all_datasets.xlsx


# Freezing 1st layer

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_1layer/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=3
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 8
Accuracy (with TN): 0.86
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): ['747072337-800x600.jpeg', '747072340-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072324-800x600.jpeg']
True Negatives (TN): 20
------------------------------
Label: washroo

2024-12-01 09:21:47.459 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: entrance
Image Count in Train: 0
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072339-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: close gym
Image Count in Train: 8
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: TV
Image Count in Train: 25
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN):

2024-12-01 09:21:52.475 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072578-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Label: swimming pool
Image Count in Train: 7
Image Count in Test: 3
Retrieved Image Count: 1
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747072587-800x600.jpeg', '747072581-800x600.jpeg']
True Negatives (TN): 9
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 0.83
False Positives (FP): ['747072583-800x600.jpeg']
False Negatives (FN): ['747072579-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 6
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072583-800x600.jpeg']
True Negatives (TN): 5
----

...

Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.85
False Positives (FP): []
False Negatives (FN): ['747073913-800x600.jpeg', '747073921-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.86
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 4
Accuracy (with TN): 0.92
False Positives (FP): ['747074038-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074039-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: swimming pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
--------

2024-12-01 09:22:43.673 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: TV
Image Count in Train: 25
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747074039-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Label: pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Label: exterior
Image Count in Train: 35
Image Count in Test: 7
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074038-800x600.jpeg', '747074042-800x600.jpeg']
True Negatives (TN): 5
------------------------------
Label: window
Image Count in Train: 41
Image Count in Test: 3
Retrieved Image Count: 3
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 9
------------------------------
Label: bed
Image Count in Train: 24
Image Count in Test: 1
Retrie

2024-12-01 09:22:46.770 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: living room
Image Count in Train: 34
Image Count in Test: 3
Retrieved Image Count: 2
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074066-800x600.jpeg']
True Negatives (TN): 6
------------------------------
Label: appliance
Image Count in Train: 31
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074069-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: swimming pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 1
Retrieved Image Count: 2
Accuracy (with TN): 0.89
False Positives (FP): ['747074067-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 7
------------------------------
Label: surrounding
Image Count in Train: 33
Image 

# Freezing 2 layer

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_2layer/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=3
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 9
Accuracy (with TN): 0.91
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): ['747072340-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 20
------------------------------
Label: washroom
Image Count in Train: 13
Image Count in Test: 1


2024-12-01 09:26:00.391 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072579-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 6
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072583-800x600.jpeg']
True Negatives (TN): 5
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747072578-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: sofa
Image Count in Train: 52
Image Count in Tes

2024-12-01 09:26:09.984 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: kid's playground
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073239-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 4
Retrieved Image Count: 6
Accuracy (with TN): 0.82
False Positives (FP): ['747073236-800x600.jpeg', '747073238-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 5
------------------------------
Label: bed
Image Count in Train: 24
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: exterior
Image Count in Train: 35
Image Count in Test: 5
Retrieved Image Count: 5
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 6
------------------------------


...

Label: play room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 12
------------------------------


2024-12-01 09:26:47.109 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 3
Accuracy (with TN): 0.85
False Positives (FP): ['747073920-800x600.jpeg']
False Negatives (FN): ['747073922-800x600.jpeg']
True Negatives (TN): 9
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 1
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073913-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.89
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): ['747074038-800x600.jpeg', '747074042-800x600.jpeg']
False Negatives (F

2024-12-01 09:26:54.480 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: living room
Image Count in Train: 34
Image Count in Test: 3
Retrieved Image Count: 2
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074066-800x600.jpeg']
True Negatives (TN): 6
------------------------------
Label: appliance
Image Count in Train: 31
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074069-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: swimming pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 1
Retrieved Image Count: 2
Accuracy (with TN): 0.89
False Positives (FP): ['747074067-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 7
------------------------------
Label: surrounding
Image Count in Train: 33
Image 

# Freezing 10 layer

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_10layer/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=3
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743


2024-12-01 09:27:36.564 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072330-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 0.91
False Positives (FP): ['747072344-800x600.jpeg']
False Negatives (FN): ['747072336-800x600.jpeg']
True Negatives (TN): 20
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 7
Accuracy (with TN): 0.82
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): ['747072340-800x600.jpeg', '747072337-800x600.jpeg', '747072334-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negativ

2024-12-01 09:27:49.456 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: entrance
Image Count in Train: 0
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072339-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 2
Accuracy (with TN): 0.95
False Positives (FP): ['747072325-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 20
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: close gym
Image Count in Train: 8
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: TV
Image Count in Train: 25
Image Count in Test: 1
Retrieved Image Coun

...

Label: play room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 12
------------------------------


2024-12-01 09:28:41.949 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 4
Accuracy (with TN): 0.77
False Positives (FP): ['747073913-800x600.jpeg', '747073920-800x600.jpeg']
False Negatives (FN): ['747073922-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.88
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 3
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 9
------------------

2024-12-01 09:28:44.924 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: lobby
Image Count in Train: 17
Image Count in Test: 2
Retrieved Image Count: 0
Accuracy (with TN): 0.83
False Positives (FP): []
False Negatives (FN): ['747074040-800x600.jpeg', '747074033-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 7
Retrieved Image Count: 7
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 5
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 3
Accuracy (with TN): 0.75
False Positives (FP): ['747074040-800x600.jpeg', '747074033-800x600.jpeg']
False Negatives (FN): ['747074032-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 3
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): ['747074033-800x600.jpeg', '747074040-800x600.jpeg']
False Negatives (FN): []
True Nega

# NCEloss

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_NCEloss/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=3
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743


2024-12-01 09:32:35.654 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072330-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 7
Accuracy (with TN): 0.82
False Positives (FP): ['747072335-800x600.jpeg']
False Negatives (FN): ['747072340-800x600.jpeg', '747072337-800x600.jpeg', '747072334-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 20
------------------------------


2024-12-01 09:32:38.032 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: washroom
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072330-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: parking
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------


...

Label: play room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------


2024-12-01 09:33:41.933 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073919-800x600.jpeg']
True Negatives (TN): 12
------------------------------


2024-12-01 09:33:42.280 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 3


Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 4
Accuracy (with TN): 0.77
False Positives (FP): ['747073913-800x600.jpeg', '747073920-800x600.jpeg']
False Negatives (FN): ['747073922-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.87
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 5
Accuracy (with TN): 0.83
False Positives (FP): ['747074038-800x600.jpeg', '747074042-800x600.jpeg']
False Negatives 

# Normal with batch 2

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch2/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=4
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743


2024-12-01 09:36:56.961 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072330-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 3
Accuracy (with TN): 0.91
False Positives (FP): ['747072344-800x600.jpeg', '747072334-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 19
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 9
Accuracy (with TN): 0.82
False Positives (FP): ['747072338-800x600.jpeg', '747072335-800x600.jpeg']
False Negatives (FN): ['747072340-800x600.jpeg', '747072334-800x600.jpeg']
True Negatives (TN): 11
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 0.91
False Positives (FP): ['747072329-800x600.jpeg']
False Negat

2024-12-01 09:36:59.142 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: washroom
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.95
False Positives (FP): []
False Negatives (FN): ['747072330-800x600.jpeg']
True Negatives (TN): 21
------------------------------
Label: parking
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------


...

Label: tredmil
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074068-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: furnishing
Image Count in Train: 77
Image Count in Test: 5
Retrieved Image Count: 6
Accuracy (with TN): 0.89
False Positives (FP): ['747074068-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 3
------------------------------


2024-12-01 09:38:11.338 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: kitchen
Image Count in Train: 28
Image Count in Test: 1
Retrieved Image Count: 5
Accuracy (with TN): 0.56
False Positives (FP): ['747074071-800x600.jpeg', '747074067-800x600.jpeg', '747074066-800x600.jpeg', '747074068-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 4
------------------------------
Label: window
Image Count in Train: 41
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074068-800x600.jpeg']
True Negatives (TN): 8
------------------------------


2024-12-01 09:38:11.664 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: top view
Image Count in Train: 2
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.89
False Positives (FP): []
False Negatives (FN): ['747074070-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Total Accuracy (with TN): 0.84
All results written to CLIP_fine-tunned_batch2_all_datasets.xlsx


# Normal with batch 16

In [ ]:
model_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch16/'

In [ ]:
# Initialize the list to collect all results
all_results = []

# Collect all unique labels across all datasets
all_labels = set()

for test_subdir in test_list_path:
    print(f"Testing {test_subdir}")
    model_name = model_dir.split('/')[-2]
    test_image_path = f'/content/drive/My Drive/Building_images_test_case/{test_subdir}'
    label_to_images, unique_labels = load_test_labels(f'{test_image_path}/test_labels.json')
    model = CLIPModel.from_pretrained(model_dir)
    processor = CLIPProcessor.from_pretrained(model_dir)

    # Call the modified test_result function and collect the results
    row_data = test_result(
        unique_labels=unique_labels,
        label_to_images=label_to_images,
        model=model,
        processor=processor,
        test_image_path=test_image_path,
        model_name=model_name,
        test_dataset_name=test_subdir,
        dynamic_threshold=4
    )
    all_results.append(row_data)  # Add the row data to the list

    # Update the set of all labels
    all_labels.update(unique_labels)
    print("=" * 150)
    print("=" * 150)
    print("=" * 150)

# After processing all datasets, create a DataFrame
df = pd.DataFrame(all_results)

# Ensure all labels are included as columns in the DataFrame
for label in all_labels:
    if label not in df.columns:
        df[label] = None  # Set missing labels to None

# Reorder columns to have 'Model Name', 'Test Dataset', labels, and 'Total Accuracy'
columns_order = ['Model Name', 'Test Dataset'] + sorted(all_labels) + ['Total Accuracy']
df = df.reindex(columns=columns_order)

# Replace NaN values with 'None' as a string
df = df.fillna('None')

# Write the DataFrame to Excel
output_excel_path = f'{model_name}_all_datasets.xlsx'
df.to_excel(output_excel_path, index=False)
print(f"All results written to {output_excel_path}")


Testing 10130743
Label: shower
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: lobby
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 21
------------------------------
Label: surrounding
Image Count in Train: 33
Image Count in Test: 9
Retrieved Image Count: 12
Accuracy (with TN): 0.86
False Positives (FP): ['747072338-800x600.jpeg', '747072335-800x600.jpeg', '747072343-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: dining
Image Count in Train: 20
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 20
------------------------------
Label: washroom
Image Count in Trai

2024-12-01 09:41:50.893 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: kid's playground
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073239-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: interior
Image Count in Train: 96
Image Count in Test: 4
Retrieved Image Count: 6
Accuracy (with TN): 0.82
False Positives (FP): ['747073236-800x600.jpeg', '747073238-800x600.jpeg']
False Negatives (FN): []
True Negatives (TN): 5
------------------------------
Label: bed
Image Count in Train: 24
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: exterior
Image Count in Train: 35
Image Count in Test: 5
Retrieved Image Count: 5
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 6
------------------------------


2024-12-01 09:41:52.022 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: balcony
Image Count in Train: 17
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.91
False Positives (FP): []
False Negatives (FN): ['747073235-800x600.jpeg']
True Negatives (TN): 10
------------------------------
Label: living room
Image Count in Train: 34
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------
Label: close gym
Image Count in Train: 8
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 10
------------------------------


...

Label: game
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: sofa
Image Count in Train: 52
Image Count in Test: 5
Retrieved Image Count: 3
Accuracy (with TN): 0.85
False Positives (FP): []
False Negatives (FN): ['747073921-800x600.jpeg', '747073922-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bathroom
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 12
------------------------------
Label: pool
Image Count in Train: 7
Image Count in Test: 1
Retrieved Image Count: 4
Accuracy (with TN): 0.62
False Positives (FP): ['747073910-800x600.jpeg', '747073918-800x600.jpeg', '747073914-800x600.jpeg', '747073915-800x600.jpeg']
False Negatives (FN): ['747073920-800x600.j

2024-12-01 09:42:27.363 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: play room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: bath
Image Count in Train: 13
Image Count in Test: 1
Retrieved Image Count: 1
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 12
------------------------------


2024-12-01 09:42:28.030 | INFO     | __main__:search_image_finetuned:66 - All the images are bellow threshold, fix_threshold: 0.23, dynamic threshold: 4


Label: game room
Image Count in Train: 1
Image Count in Test: 1
Retrieved Image Count: 0
Accuracy (with TN): 0.92
False Positives (FP): []
False Negatives (FN): ['747073911-800x600.jpeg']
True Negatives (TN): 12
------------------------------
Label: terace
Image Count in Train: 17
Image Count in Test: 3
Retrieved Image Count: 4
Accuracy (with TN): 0.77
False Positives (FP): ['747073913-800x600.jpeg', '747073920-800x600.jpeg']
False Negatives (FN): ['747073922-800x600.jpeg']
True Negatives (TN): 8
------------------------------
Label: bedroom
Image Count in Train: 23
Image Count in Test: 2
Retrieved Image Count: 2
Accuracy (with TN): 1.00
False Positives (FP): []
False Negatives (FN): []
True Negatives (TN): 11
------------------------------
Total Accuracy (with TN): 0.89
Testing 10130846
Label: view
Image Count in Train: 37
Image Count in Test: 3
Retrieved Image Count: 4
Accuracy (with TN): 0.92
False Positives (FP): ['747074042-800x600.jpeg']
False Negatives (FN): []
True Negatives (T